In [1]:
import pandas as pd
import pickle

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv', encoding='ISO-8859-1')

In [3]:
train['education'] = train['Have you Completed your Graduation ?'] + ' ' + train['Highest Educational Qualification']
train = train.drop(['Have you Completed your Graduation ?', 'Highest Educational Qualification', 'Residential Pincode', 'Branch Pincode'], axis=1)
test['education'] = test['Have you Completed your Graduation ?'] + ' ' + test['Highest Educational Qualification']
test = test.drop(['Have you Completed your Graduation ?', 'Highest Educational Qualification', 'Residential Pincode', 'Branch Pincode'], axis=1)

In [4]:
#dont execute this
columns_to_drop = ['How many Organization that you have worked before joining Piramal Finance ?',
                  'CandidateID', 'Designation', 'DOJ', 'Location Code']
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)

In [4]:
with open("resume_data.pkl", "rb") as file:
    resume_text_new = pickle.load(file)

In [5]:
resume_txt = pd.DataFrame(resume_text_new, columns=['resume_txt'])
df = pd.concat([train, resume_txt], axis=1)

In [6]:
#optional
with open("resume_data_test.pkl", "rb") as file:
    resume_text_test = pickle.load(file)

In [7]:
with open("resume_data_test_new1.pkl", "rb") as file:
    resume_text_test1 = pickle.load(file)

In [8]:
with open("resume_data_test_new2.pkl", "rb") as file:
    resume_text_test2 = pickle.load(file)

In [7]:

#resume_text_test = resume_text_test1 + resume_text_test2
resume_test = pd.DataFrame(resume_text_test, columns = ['resume_txt'])

In [8]:
test = pd.concat([test, resume_test], axis=1)

In [9]:
df.to_csv('train1.csv', index=False)
test.to_csv('test1.csv', index=False)

In [10]:
from autogluon.core.dataset import TabularDataset
train_data = TabularDataset('train1.csv')
train_data.head(30)
train_data.columns

Index(['CandidateID', 'Designation',
       'Total no of years Experience [before joining Piramal]',
       'Previous Industry worked with [before joining Piramal]',
       'Name of your Previous Organization / Company',
       'How many Organization that you have worked before joining Piramal Finance ?',
       'Average Incentive [per month] earned in your pervious company ?',
       'How did you come to know about the role at Piramal Finance ?',
       'Which Products you are selling in your pervious role ?',
       'What was the average ticket size handled at your end in previous role ?',
       'How many members are there in your family ?',
       'How many are earning family members ? [Other then yourself]2',
       'How many members are dependent on you ?', 'Department', 'DOJ',
       'Location Code', 'Performance', 'education', 'resume_txt'],
      dtype='object')

In [11]:
from autogluon.tabular import TabularPredictor 
predictor = TabularPredictor(label='Performance').fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20240410_185109"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240410_185109"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:     

In [12]:
test_data = TabularDataset('test1.csv')
label = 'CandidateID'
y_pred = predictor.predict((test_data))
y_pred.head()

Loaded data from: test1.csv | Columns = 18 / 18 | Rows = 187 -> 187


0    1
1    1
2    0
3    0
4    0
Name: Performance, dtype: int64

In [13]:
sub=pd.DataFrame()
sub['CandidateID'],sub['Performance']=test_data['CandidateID'],y_pred
sub.to_csv('submission_3.csv', index=False)
sub

,CandidateID,Performance
0,EMP0521,1
1,EMP0613,1
2,EMP0136,0
3,EMP0351,0
4,EMP0049,0
...,...,...
182,EMP0401,1
183,EMP0408,0
184,EMP0248,0
185,EMP0148,0
